# Bank marketing data

This is an exercise with the following instructions:

A csv file called `"bank_marketing.csv"` has been supplied to you, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [ ]:
import pandas as pd
import numpy as np

raw_df = pd.read_csv('data/bank_marketing.csv')
print(raw_df.info())

In [ ]:
# Create the subsets
subsets = [['client_id', 'age', 'job', 'marital', 'education', 'credit_default', 'mortgage'],
           ['client_id', 'number_contacts', 'contact_duration', 'previous_campaign_contacts', 'previous_outcome', 'campaign_outcome', 'month', 'day'],
           ['client_id', 'cons_price_idx', 'euribor_three_months']]

split_data = []

for subset in subsets:
    selection = raw_df.loc[:, subset]
    split_data.append(selection)

print(len(split_data))

In [ ]:
# Dataframe inspection
for i in range(3):
    print(split_data[i].info())
    print()

-----------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   client_id       41188 non-null  int64 
 1   age             41188 non-null  int64 
 2   job             41188 non-null  object
 3   marital         41188 non-null  object
 4   education       41188 non-null  object
 5   credit_default  41188 non-null  object
 6   mortgage        41188 non-null  object
dtypes: int64(2), object(5)
memory usage: 2.2+ MB
None
-----------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   client_id                   41188 non-null  int64 
 1   number_contacts             41188 non-null  int64 
 2   contact_duration            4118

In [ ]:
# Client dataframe cleaning
def clean_client(data):
    df = data.copy()
    
    # job: change '.' to '_'
    df['job'] = df['job'].str.replace('.', '_')
    df['job'] = df['job'].str.strip('_')
    # education: change '.' to '_' and add NaN
    df['education'] = df['education'].str.replace('.', '_')
    df['education'] = df['education'].apply(lambda x: np.nan if x=='unknown' else x)
    # credit default: 1 if yes, else 0, dtype is bool
    df['credit_default'] = df['credit_default'].apply(lambda x: 1 if x=='yes' else 0)
    df['credit_default'] = df['credit_default'].astype('bool')
    # mortgage: 1 if yes, else 0, dtype is bool
    df['mortgage'] = df['mortgage'].apply(lambda x: 1 if x=='yes' else 0)
    df['mortgage'] = df['mortgage'].astype('bool')
    
    return df

client = clean_client(split_data[0])
print(client.info())
client.to_csv('./data/client.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   client_id       41188 non-null  int64 
 1   age             41188 non-null  int64 
 2   job             41188 non-null  object
 3   marital         41188 non-null  object
 4   education       39457 non-null  object
 5   credit_default  41188 non-null  bool  
 6   mortgage        41188 non-null  bool  
dtypes: bool(2), int64(2), object(3)
memory usage: 1.6+ MB
None


In [ ]:
# Campaign dataframe cleaning
def clean_campaign(data):
    df = data.copy()
    
    # previous outcome: 1 if success else 0, dtype is bool
    df['previous_outcome'] = df['previous_outcome'].apply(lambda x: 1 if x=='success' else 0)
    df['previous_outcome'] = df['previous_outcome'].astype('bool')
    # campaign outcome: 1 if yes else 0, dtype is bool
    df['campaign_outcome'] = df['campaign_outcome'].apply(lambda x: 1 if x=='yes' else 0)
    df['campaign_outcome'] = df['campaign_outcome'].astype('bool')
    # year column: 2022
    df['year'] = '2022'
    # day column to str to concatenate
    df['day'] = df['day'].astype('str')
    # create last_contact_date column
    df['last_contact_date'] = df['year'] + '-' + df['month'] + '-' + df['day']
    df['last_contact_date'] = pd.to_datetime(df['last_contact_date'], format='%Y-%b-%d')
    # drop the columns used to create dates
    df.drop(['year', 'day', 'month'], axis=1, inplace=True)
    
    return df

campaign = clean_campaign(split_data[1])
print(campaign.info())
campaign.to_csv('./data/campaign.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   client_id                   41188 non-null  int64         
 1   number_contacts             41188 non-null  int64         
 2   contact_duration            41188 non-null  int64         
 3   previous_campaign_contacts  41188 non-null  int64         
 4   previous_outcome            41188 non-null  bool          
 5   campaign_outcome            41188 non-null  bool          
 6   last_contact_date           41188 non-null  datetime64[ns]
dtypes: bool(2), datetime64[ns](1), int64(4)
memory usage: 1.6 MB
None


In [ ]:
# Economics dataframe cleaning: just save as csv
split_data[2].to_csv('./data/economics.csv', index=False)